In [1]:
%load_ext autoreload
%load_ext autotime

time: 182 µs (started: 2024-10-09 15:45:21 -07:00)


In [2]:
import os
import sys
import time
import warnings

# Add WisIO to the path
sys.path.append(os.path.abspath("../"))

# Filter warnings
warnings.filterwarnings('ignore')

time: 537 µs (started: 2024-10-09 15:45:21 -07:00)


In [3]:
analyzer = "dftracer"
checkpoint = True
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = f".wisio/{analyzer}/{int(time.time())}"
bottleneck_dir = f"{run_dir}/bottlenecks"
checkpoint_dir = f"{run_dir}/checkpoints"
percentile = 0.95
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/cm1/APP/node-1/v1/RAW/*.pfw.gz"

time: 751 µs (started: 2024-10-09 15:45:21 -07:00)


In [4]:
# Parameters
analyzer = "dftracer"
checkpoint = True
checkpoint_dir = "/usr/workspace/iopp/wisio_logs/_checkpoints/dftracer-cosmoflow-dlio-v100-node-16-v3"
logical_view_types = False
output_max_bottlenecks = 1
output_root_only = False
run_dir = ".wisio/dftracer/cosmoflow-dlio-v100-node-16-v3"
percentile = 0.99
trace_path = "/p/lustre3/iopp/dftracer-traces-lfs/v1.0.5-develop/corona/cosmoflow/dlio-v100/node-16/v3/COMPACT/*.pfw.gz"


time: 659 µs (started: 2024-10-09 15:45:21 -07:00)


In [5]:
from distributed import Client
from hydra import compose, initialize
from hydra.core.hydra_config import HydraConfig
from hydra.utils import instantiate
from wisio.config import init_hydra_config_store
from wisio.utils.file_utils import ensure_dir

ensure_dir(run_dir)

with initialize(version_base=None, config_path=None):
    init_hydra_config_store()
    cfg = compose(
        config_name="config",
        overrides=[
            f"+analyzer={analyzer}",
            f"analyzer.bottleneck_dir={bottleneck_dir}",
            f"analyzer.checkpoint={checkpoint}",
            f"analyzer.checkpoint_dir={checkpoint_dir}",
            f"hydra.run.dir={run_dir}",
            f"hydra.runtime.output_dir={run_dir}",
            f"logical_view_types={logical_view_types}",
            f"output.max_bottlenecks={output_max_bottlenecks}",
            f"output.root_only={output_root_only}",
            f"percentile={percentile}",
            f"trace_path={trace_path}",
        ],
        return_hydra_config=True,
    )
    HydraConfig.instance().set_config(cfg)
    cluster = instantiate(cfg.cluster)
    client = Client(cluster)
    analyzer = instantiate(cfg.analyzer)
    output = instantiate(cfg.output)

time: 7.43 s (started: 2024-10-09 15:45:21 -07:00)


In [6]:
result = analyzer.analyze_trace(
    trace_path=cfg.trace_path,
    # accuracy=cfg.accuracy,
    exclude_bottlenecks=cfg.exclude_bottlenecks,
    exclude_characteristics=cfg.exclude_characteristics,
    logical_view_types=cfg.logical_view_types,
    metrics=cfg.metrics,
    percentile=cfg.percentile,
    view_types=cfg.view_types,
)

time: 36min 22s (started: 2024-10-09 15:45:28 -07:00)


In [7]:
output.handle_result(metrics=cfg.metrics, result=result)

╭────────────────────────────────────────────── I/O Characteristics ──────────────────────────────────────────────╮
│                                                                                                                 │
│  Runtime          769.93 seconds                                                                                │
│  I/O Time         34.48 seconds                                                                                 │
│                   ├── Read - 32.23 seconds (93.47%)                                                             │
│                   ├── Write - 0.00 seconds (0.01%)                                                              │
│                   └── Metadata - 2.60 seconds (7.55%)                                                           │
│  I/O Operations   45,802,980 ops                                                                                │
│                   ├── Read - 32,131,265 ops (70.15%)                                                            │
│                   ├── Write - 4,217 ops (0.01%)                                                                 │
│                   └── Metadata - 13,632,170 ops (29.76%)                                                        │
│  I/O Size         6.76 TiB                                                                                      │
│                   ├── Read - 6.76 TiB (100.00%)                                                                 │
│                   └── Write - 218.81 kiB (0.00%)                                                                │
│  Read Requests    4 kiB-256 kiB - 32,130,509 ops                                                                │
│                   ├── <4 kiB - 175,248 ops (0.55%)                                                              │
│                   ├── 4-16 kiB - 20,092 ops (0.06%)                                                             │
│                   ├── 16-64 kiB - 31,444 ops (0.10%)                                                            │
│                   └── 64-256 kiB - 31,903,725 ops (99.29%)                                                      │
│  Write Requests   4 kiB-4 kiB - 4,217 ops                                                                       │
│                   └── <4 kiB - 4,217 ops (100.00%)                                                              │
│  Nodes            16 nodes                                                                                      │
│                   ├── 3903101653980564509 - 34.48 s (100.00%) - 432.81 GiB/10.56 kiB R/W (6.25/0.00%) -         │
│                   │   2,862,327 ops (6.25%)                                                                     │
│                   ├── 16317731584986882421 - 34.10 s (98.88%) - 432.80 GiB/60.38 kiB R/W (6.25/0.00%) -         │
│                   │   2,862,635 ops (6.25%)                                                                     │
│                   ├── 17424505939907203644 - 33.81 s (98.07%) - 432.84 GiB/10.56 kiB R/W (6.25/0.00%) -         │
│                   │   2,862,193 ops (6.25%)                                                                     │
│                   ├── 3331529830307220231 - 33.52 s (97.23%) - 432.91 GiB/10.56 kiB R/W (6.25/0.00%) -          │
│                   │   2,863,177 ops (6.25%)                                                                     │
│                   ├── 11940681009529776706 - 33.47 s (97.06%) - 432.79 GiB/10.56 kiB R/W (6.25/0.00%) -         │
│                   │   2,862,420 ops (6.25%)                                                                     │
│                   ├── 9984750283406985853 - 32.90 s (95.43%) - 432.78 GiB/10.56 kiB R/W (6.25/0.00%) -          │
│                   │   2,862,194 ops (6.25%)                                                                     │
│                   ├── 13943161580101246997 - 32.81 s (

time: 5min 35s (started: 2024-10-09 16:21:50 -07:00)
